# This notebook is to replicate what we were able to achieve on the AWS console

- Creating through bedrock-agent for more confirugation and control

In [1]:
import boto3
import sys
import json
import pprint
from botocore.client import Config
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
import os
import random
from retrying import retry
import time
from utility import *
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.35.34


In [2]:
# Create bedrock agent client
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name or 'ca-central-1'
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime", config=bedrock_config)

# Define FM to be used for generations 
model_id = "anthropic.claude-3-sonnet-20240229-v1:0" 
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

# knowledge-base-AWS-bedrock-psychiatrist
kb_id = '76UIT87ACB'

In [4]:

# R and G is to `retrieve_and_generate`
def retrieve_and_generate(query, kb_id, model_arn, max_results, prompt_template):
    response = bedrock_agent_client.retrieve_and_generate(
            input={
                'text': query
            },
        retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn,

            # Retrieval Configuration
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results,
                    'overrideSearchType': 'HYBRID' or 'SEMANTIC'
                    }
                }, 

            # Generation Configuration
            'generationConfiguration': {
                    'promptTemplate': {
                        'textPromptTemplate': prompt_template
                    } ,
                    'inferenceConfig': {
                    'textInferenceConfig': {
                        'maxTokens': 512,
                        'temperature': 1.0,
                        'topP': 1.0
                        }
                    }
                },

            # Orchestration Confirugation
            'orchestrationConfiguration': {
                'queryTransformationConfiguration': {
                    'type': 'QUERY_DECOMPOSITION'
                    }
                } 
            
            } # Knowledge base configuration
        }
    )
    return response


# Printing Generation Results refers to the citations / references from the KB
def print_generation_results(query, response, print_context = False):
    generated_text = response['output']['text']
    print (query)
    print(generated_text)
    
    if print_context is True:
        ## print out the source attribution/citations from the original documents to see if the response generated belongs to the context.
        citations = response["citations"]
        contexts = []
        for citation in citations:
            retrievedReferences = citation["retrievedReferences"]
            for reference in retrievedReferences:
                contexts.append(reference["content"]["text"])
    
        print('\n\n\nRetrieved Context:\n')
        pprint.pp(contexts)


# Test with potential questions

In [16]:
# Assigning the default knowledge base prompt

path = '../machine-learning/4. Knowledge Base Template/'
# filename = 'RoleSwitching_V4_Oct2.md'
filename = 'App_Assistant_V1_Oct10.md'

with open(path+filename, 'r') as f:
    prompt_template = f.read()
    

In [22]:
path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions2.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')

for i in queries:
    response = retrieve_and_generate(query = i, kb_id = kb_id, model_arn = model_arn, max_results = 10, prompt_template = prompt_template)
    print_generation_results(i, response)
    print ('---------------------------------------------------- \n')
    

Is it possible to customize the app without logging in?
Yes, you can customize and personalize the Manage My Pain app without logging in or creating an account. When you first open the app, there are introduction screens that explain the app's features. You can then proceed to accept the End User License Agreement and select your pain conditions and medications during the initial setup process, all before creating an account or logging in.
---------------------------------------------------- 

Am I required to accept the End User License Agreement?
Yes, accepting the End User License Agreement is required to use Manage My Pain.
---------------------------------------------------- 

Why isn’t my specific pain condition listed?
If you cannot find your specific pain condition listed in the app, you can add a new one by following these steps:

1. From the "Add Pain Condition" screen, type 3 or more letters of your condition and wait for the search results.
2. If your condition is not found

In [18]:
path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions3.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')

for i in queries:
    response = retrieve_and_generate(query = i, kb_id = kb_id, model_arn = model_arn, max_results = 10, prompt_template = prompt_template)
    print_generation_results(i, response)
    print ('---------------------------------------------------- \n')
    

Will any of my personal information be shared?
No, your personal information will not be shared with anyone. As stated in the privacy policy, ManagingLife takes security and privacy very seriously. Your data is protected through measures such as encrypted data transmission, restricted access to data, and secure storage on AWS servers.
---------------------------------------------------- 

Do I need to use the same email for both my Google and Facebook accounts to link them in the app?
No, you do not need to have the same email address for your Google and Facebook accounts to connect them in the Manage My Pain app. However, a Google account and a Facebook account can only be connected to a single account in the app.
---------------------------------------------------- 

Is setting a password required when signing up with Google or Facebook?
No, it is not required to set a password when registering via Google or Facebook. However, you have the option to set a password so that you can log

In [21]:
path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions4-Unrelated.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')

for i in queries:
    response = retrieve_and_generate(query = i, kb_id = kb_id, model_arn = model_arn, max_results = 10, prompt_template = prompt_template)
    print_generation_results(i, response)
    print ('---------------------------------------------------- \n')
    

What is the weather like today?
Sorry, I am unable to assist you with this request.
---------------------------------------------------- 

How do I start learning a new language?
Sorry, I am unable to assist you with this request.
---------------------------------------------------- 

What are the benefits of a healthy diet?
Sorry, I am unable to assist you with this request.
---------------------------------------------------- 

How do I improve my time management skills?
The Manage My Pain app can help you improve your time management skills by allowing you to track your daily activities and prioritize meaningful activities. You can use the app to:

1. Set reminders to log your pain levels and activities at specific times each day.
2. Plan your daily schedule around your best times of day based on when your pain is lowest.
3. Break down larger tasks into smaller, more manageable steps and alternate between easier and harder activities using the activity shifting strategy. Additionall